###  다음 분기에 어떤 게임을 설계해야 할까 - 전처리 


In [ ]:
from google.colab import files
myfile = files.upload()

In [ ]:
#io와 pandas 모듈 import
import io
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv(io.BytesIO(myfile['vgames2.csv']))
df = df.iloc[:,1:]
df.head(12)

In [ ]:
df.shape

In [ ]:
df.isnull().sum()
df.info()

### 출고량 단위 통일

In [ ]:
pdf = df.copy()
pdf.dtypes

In [ ]:
#출고량 단위 통일
def change(x):
    lst = pdf[x].to_list()
    ssu = []
    for i in lst:
        if i[-1] == 'K':
            ssu.append(float(i[:-1])*0.001)
        elif i[-1] == 'M':
            ssu.append(float(i[:-1]))
        else:
            ssu.append(float(i))
    return pd.DataFrame(ssu)

In [ ]:
pdf['NA_Sales'] = change('NA_Sales')
pdf['EU_Sales'] = change('EU_Sales')
pdf['JP_Sales'] = change('JP_Sales')
pdf['Other_Sales'] = change('Other_Sales')

In [ ]:
col = ['Name', 'Platform','Year','Genre', 'Publisher', 'NA_Sales (unit:M)',	'EU_Sales (unit:M)','JP_Sales (unit:M)'	,'Other_Sales (unit:M)']
pdf.columns = col
pdf.head(12)

In [ ]:
# def prefixFinder(x):

#     if x.endswith('k'):
#         x = x[:-1]               
#         return float(x) * 1000 


### 지역에 따라 선호하는 장르 df1


In [ ]:
#지역별로 장르 1순위 고르는거?? 어떻게 하면 좋을까??
pdf1 = pdf[['Genre','NA_Sales (unit:M)','EU_Sales (unit:M)','JP_Sales (unit:M)','Other_Sales (unit:M)']] 
#.set_index('Genre') 이렇게 해버리면 groupby 할 때 적용이 안됨

pdf1

In [ ]:
# df1['NA_Sales (unit:M)'].idxmax()
# df1['EU_Sales (unit:M)'].idxmax()
# df1['JP_Sales (unit:M)'].idxmax()
# df1['Other_Sales (unit:M)'].idxmax()

In [ ]:
grouped = pdf1.groupby(by = ['Genre'], as_index = True).sum()#['NA_Sales (unit:M)'] 컬럼 선택해서 보기~
grouped

In [ ]:
# sdfd = grouped['NA_Sales (unit:M)'].nlargest(1)
# sdfd

In [ ]:
#지역별 판매울 높은 게임장르
max = []
for i in grouped:
    max.append(grouped[i].idxmax())
    #max.append(grouped[i].nlargest(1))
maxx = pd.DataFrame(max).T
maxx.columns = grouped.columns
maxx

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt


f, ax= plt.subplots(1, 2, sharey=True)
# colors = ['darkkhaki', 'darkorange', 'cadetblue','royalblue']
# grouped.plot.bar(rot=0,width=0.8,figsize=(8,8), color = colors);

plt.rcParams["figure.figsize"] = (24,10)
sns.barplot(x= grouped.index, y='NA_Sales (unit:M)', data=grouped, color='#99c2a2',ax=ax[0])
sns.barplot(x= grouped.index, y='EU_Sales (unit:M)', data=grouped, color='#99c2a2',ax=ax[1])
plt.show();

In [ ]:
f, ax= plt.subplots(1, 2, sharey=True)

plt.rcParams["figure.figsize"] = (24,10)
sns.barplot(x= grouped.index, y='JP_Sales (unit:M)', data=grouped, color='#99c2a2',ax=ax[0])
sns.barplot(x= grouped.index, y='Other_Sales (unit:M)', data=grouped, color='#99c2a2',ax=ax[1])
plt.show();

### 판매량 지역순위


In [ ]:
pdf11 = pdf[['Genre','NA_Sales (unit:M)','EU_Sales (unit:M)','JP_Sales (unit:M)','Other_Sales (unit:M)']]
pdf11 = pdf11.set_index('Genre').sum()

In [ ]:
pdf11 = pd.DataFrame(pdf11, columns=['Sales'])
pdf11

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
#plt.figure(figsize=(20,10))
plt.rcParams["figure.figsize"] = (10,6)
ax = sns.barplot(x = pdf11.index, y = pdf11['Sales'], data=pdf11, palette='coolwarm')

plt.xlabel('Local')#,fontsize = 20)
plt.ylabel("volum")#,fontsize = 20
#plt.xticks(fontsize = 20)
plt.title('Local sales volum')#, fontsize=30)
#plt.xticks(rotation = 20) 각도 조절을 해도 이름이 다 들어오지 않아 x,y축 변경
plt.show();

# 어느지역에 출고해야할까??
### ANOVA Test

In [ ]:
#'NA_Sales (unit:M)','EU_Sales (unit:M)','JP_Sales (unit:M)','Other_Sales (unit:M)'
NA = grouped.loc[:,['NA_Sales (unit:M)']]
EU = grouped.loc[:,['EU_Sales (unit:M)']]
JP = grouped.loc[:,['JP_Sales (unit:M)']]
Ot = grouped.loc[:,['Other_Sales (unit:M)']]
Global = [NA, EU, JP, Ot]
Global

In [ ]:
# Kernel Density Estimate Plot
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['figure.figsize'] = [10, 6]

sns.kdeplot(grouped['NA_Sales (unit:M)'])
sns.kdeplot(grouped['EU_Sales (unit:M)'])
sns.kdeplot(grouped['JP_Sales (unit:M)'])
sns.kdeplot(grouped['Other_Sales (unit:M)'])
plt.show()

In [ ]:
# Boxplot
sns.boxplot(data=Global)
plt.xlabel("Group", fontsize=14)
plt.ylabel("Value", fontsize=14)
plt.show()

In [ ]:
grouped.describe().T

In [ ]:
#   - 귀무가설(H0): 4 집단의 평균은 모두 같다. (mu1 = mu2 = m3 = m4)

#   - 대립가설(H1):  4 집단의 평균은 같지 않다.(적어도 1개 집단의 평균은 같지 않다) (Not H0)

In [ ]:
# ANOVA with different sized samples using scipy
from scipy import stats

stats.f_oneway(NA, EU, JP, Ot)

In [ ]:
# returning f-statistic and p-value
f_val, p_val = stats.f_oneway(*Global)

print('f-statistic:', f_val)
print('p-vale:', p_val)

#연도별 게임의 트렌드 df2

In [ ]:
#각 연도마다 가장 많은 판매량의 게임을 구하자!! 어떻게?? 일단 연도별 게임을 묶어
def n_year(number):
    if number == range(0,20):
        return number+2000

    elif number == range(86, 100):
        return number+1900
    else:
        return number
pdf['Year']=pdf['Year'].apply(n_year)   
pdf

In [ ]:
pdf.isnull().sum()

In [ ]:
#필요한 컬럼 선택하고 결측치 제거
pdf2 = pdf.iloc[:,2:4]
pdf2.dropna(inplace=True)
pdf2

In [ ]:
#연도별로 묶은 뒤 그래프그리기
pdf2_1970s = pdf2.query('Year < 1980 ')
#display_side_by_side(pdf2, pdf2_1970s)
pdf2_1970s = pd.crosstab(pdf2_1970s.Year,pdf2_1970s.Genre).sum()
pdf2_1970s = pd.DataFrame(pdf2_1970s, columns = ['1970s'])
pdf2_1970s

In [ ]:
#pdf2_1970s 시각화 하기
plt.rcParams["figure.figsize"] = (12,6)
sns.barplot(x = pdf2_1970s.index , y = pdf2_1970s['1970s'], data = pdf2_1970s)

In [ ]:
pdf2_1980s = pdf2.query('Year < 1990 ')
pdf2_1980s = pd.crosstab(pdf2_1980s.Year,pdf2_1980s.Genre).sum()
pdf2_1980s = pd.DataFrame(pdf2_1980s, columns = ['1980s'])
pdf2_1980s

In [ ]:
#pdf2_1980s 시각화 하기
plt.rcParams["figure.figsize"] = (12,6)
sns.barplot(x = pdf2_1980s.index , y = pdf2_1980s['1980s'], data = pdf2_1980s)

In [ ]:
pdf2_1990s = pdf2.query('Year < 2000 ')
pdf2_1990s = pd.crosstab(pdf2_1990s.Year,pdf2_1990s.Genre).sum()
pdf2_1990s = pd.DataFrame(pdf2_1990s, columns = ['1990s'])
pdf2_1990s

In [ ]:
#pdf2_1990s 시각화 하기
plt.rcParams["figure.figsize"] = (12,6)
sns.barplot(x = pdf2_1990s.index , y = pdf2_1990s['1990s'], data = pdf2_1990s)

In [ ]:
pdf2_2000s = pdf2.query('2000 <= Year< 2010')
pdf2_2000s = pd.crosstab(pdf2_2000s.Year,pdf2_2000s.Genre).sum()
pdf2_2000s = pd.DataFrame(pdf2_2000s, columns = ['2000s'])
pdf2_2000s

In [ ]:
#pdf2_2000s 시각화 하기
plt.rcParams["figure.figsize"] = (12,6)
sns.barplot(x = pdf2_2000s.index , y = pdf2_2000s['2000s'], data = pdf2_2000s)

In [ ]:
pdf2_2010s = pdf2.query('2010 <= Year < 2020')
pdf2_2010s = pd.crosstab(pdf2_2010s.Year,pdf2_2010s.Genre).sum()
pdf2_2010s = pd.DataFrame(pdf2_2010s, columns = ['2010s'])
pdf2_2010s

In [ ]:
#pdf2_2010s 시각화 하기
plt.rcParams["figure.figsize"] = (12,6)
sns.barplot(x = pdf2_2010s.index , y = pdf2_2010s['2010s'], data = pdf2_2010s)

# 연도별 게임 판매랑 모아서 그래프로

In [ ]:
#pd.merge(left2, right2, how="outer", left_index=True, right_index=True)

pdf2_y = pd.concat([pdf2_1970s,pdf2_1980s,pdf2_1990s,pdf2_2000s,pdf2_2010s], join='inner', axis=1)
pdf2_y = pdf2_y.T
pdf2_y

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# fig = plt.figure(figsize=(20,10)) ## 캔버스 생성
# fig.set_facecolor('white') ## 캔버스 색상 설정
# ax = fig.add_subplot() ## 그림 뼈대(프레임) 생성


pdf2_y.plot() ## 선그래프 생성
plt.xticks() ## x축 눈금 라벨 설정 - 40도 회전 
plt.title('Sales for 10 days',fontsize=20) ## 타이틀 설정
plt.xlabel('Year')
plt.ylabel('Genre')
plt.show()

In [ ]:
# ## 범례 -선 그래프 기본
# import matplotlib.pyplot as plt
 
# ## 성별 데이터 분리하기
# male_df = df.query('gender =="male"') ## 남자 데이터
# male_visits = male_df['visits'] ## 남자 방문자수
# days = male_df['days'] ## 요일
 
# female_df = df.query('gender == "female"') ## 여자 데이터
# female_visits = female_df['visits'] ## 여자 방문자수
 
# ## 선그래프 그리기
# fig = plt.figure(figsize=(8,8)) ## 캔버스 생성
# fig.set_facecolor('white') ## 캔버스 색상 설정
# ax = fig.add_subplot() ## 프레임(그림 뼈대) 생성
 
# ## 선 그래프 생성
# ax.plot(days, male_visits, marker='o', label='male') 
# ax.plot(days, female_visits, marker='o', label='female')
 
# ax.legend() ## 범례
 
# plt.show()

In [ ]:
#plt.savefig('sample.png') 시각화 자료 파일로 저장

In [ ]:
plt.figure(figsize=(15,10))
sns.lineplot(data=pdf2_y)
plt.title('Game Sales by Year',fontsize=20) ## 타이틀 설정
plt.xlabel('Year')
plt.ylabel('Genre')
plt.legend(bbox_to_anchor=(1.01, 1), loc=2, borderaxespad=0.)

In [ ]:
#sns.relplot(kind="line", data=pdf2_y)

In [ ]:
# sns.relplot(
#     kind="line", data=df_last, x="연도", y="평당분양가격", hue="지역명", 
#     col="지역명", col_wrap=4)

# 출고량이 높은 게임에 대한 분석 및 시각화 프로세스 df3

In [ ]:
#전체적으로 출고량이 높은 게임을 찾아!! 어떻게 찾을래??
pdf['Total_Sales (unit:M)'] = pdf['NA_Sales (unit:M)']+pdf['EU_Sales (unit:M)']+pdf['JP_Sales (unit:M)']+pdf['Other_Sales (unit:M)']
pdf

In [ ]:
pdf3 = pdf.loc[:,['Name','Total_Sales (unit:M)']]
pdf3 = pdf3.nlargest(10, 'Total_Sales (unit:M)')
pdf3.set_index('Name', drop=True, inplace=True)
pdf3

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
#plt.figure(figsize=(20,10))
plt.rcParams["figure.figsize"] = (10,6)
ax = sns.barplot(x = pdf3['Total_Sales (unit:M)'], y = pdf3.index, data=pdf3, palette='coolwarm')

plt.xlabel('Total_Sales (unit:M)')#,fontsize = 20)
plt.ylabel("Games")#,fontsize = 20
#plt.xticks(fontsize = 20)
plt.title('< Top 10 > Global Sales amount')#, fontsize=30)
#plt.xticks(rotation = 20) 각도 조절을 해도 이름이 다 들어오지 않아 x,y축 변경
plt.show();

In [ ]:
pdf3_g = pdf.loc[:,['Genre','Total_Sales (unit:M)']]
pdf3_g = pdf3_g.groupby(by=['Genre'], as_index = True).sum()
pdf3_g

In [ ]:
pdf3_g = pdf3_g.nlargest(10, 'Total_Sales (unit:M)')
pdf3_g

In [ ]:
plt.rcParams["figure.figsize"] = (10,6)
ax = sns.barplot(x = pdf3_g['Total_Sales (unit:M)'], y = pdf3_g.index, data=pdf3_g, palette='coolwarm')

plt.xlabel('Total_Sales (unit:M)')#,fontsize = 20)
plt.ylabel("Genre")#,fontsize = 20
#plt.xticks(fontsize = 20)
plt.title('< Top 10 > Global Sales amount')#, fontsize=30)
#plt.xticks(rotation = 20) 각도 조절을 해도 이름이 다 들어오지 않아 x,y축 변경
plt.show();

# 어떤 플랫폼을 이용할까?

In [ ]:
# xbox : X360, XB, XOne
# nintendo : Wii, WiiU, N64, GC, NES / 3DS, DS, GB, GBA, SNES
# playstation : PS, PS2, PS3, PS4 / PSP, PSV
# other : PC, 2600, SAT, DC, SNES, SCD, TG16, 3DO, PCFX, NG / GG,  WS, GEN

In [ ]:
pdf4 = pdf.loc[:,['Platform','Total_Sales (unit:M)']]
pdf4

In [ ]:
pdf4 = pdf4.groupby(by = ['Platform'], as_index = True).sum().T
pdf4

In [ ]:
pdf4['xbox'] = pdf4['X360'] + pdf4['XB'] + pdf4['XOne']
pdf4['nintendo'] = pdf4['Wii']+ pdf4['WiiU']+ pdf4['N64']+ pdf4['GC']+ pdf4['NES']+ pdf4['3DS']+ pdf4['DS']+ pdf4['GB']+ pdf4['GBA']+ pdf4['SNES']
pdf4['playstation'] = pdf4['PS']+ pdf4['PS2']+ pdf4['PS3']+ pdf4['PS4']+pdf4['PSP']+ pdf4['PSV']
pdf4['other'] = pdf4['PC']+ pdf4['2600']+ pdf4['SAT']+ pdf4['DC']+ pdf4['SNES']+ pdf4['SCD']+ pdf4['TG16']+ pdf4['3DO']+ pdf4['PCFX']+ pdf4['NG']+ pdf4['GG']+ pdf4['WS']+ pdf4['GEN']
pdf4

In [ ]:
pdf4_p = pdf4.loc[:,['xbox','nintendo','playstation','other']]
pdf4_p = pdf4_p.T
pdf4_p

In [ ]:
plt.rcParams["figure.figsize"] = (10,6)
plt.ylim(0,4000)
# ax.hlines(x=3580.82, color = 'Red')
sns.barplot(y = pdf4_p.index, x = pdf4_p['Total_Sales (unit:M)'], data = pdf4_p, palette= 'magma');

### 플랫폼 선택 추가 근거
T-test

In [ ]:
pdf14 = pdf.loc[:,['Platform','NA_Sales (unit:M)','EU_Sales (unit:M)','JP_Sales (unit:M)','Other_Sales (unit:M)']]
pdf14

In [ ]:
pdf14 = pdf14.groupby(by = ['Platform'], as_index = True).sum().T
pdf14

In [ ]:
pdf14['xbox'] = pdf14['X360'] + pdf14['XB'] + pdf14['XOne']
pdf14['nintendo'] = pdf14['Wii']+ pdf14['WiiU']+ pdf14['N64']+ pdf14['GC']+ pdf14['NES']+ pdf14['3DS']+ pdf14['DS']+ pdf14['GB']+ pdf14['GBA']+ pdf14['SNES']
pdf14['playstation'] = pdf14['PS']+ pdf14['PS2']+ pdf14['PS3']+ pdf14['PS4']+pdf14['PSP']+ pdf14['PSV']
pdf14['other'] = pdf14['PC']+ pdf14['2600']+ pdf14['SAT']+ pdf14['DC']+ pdf14['SNES']+ pdf14['SCD']+ pdf14['TG16']+ pdf14['3DO']+ pdf14['PCFX']+ pdf14['NG']+ pdf14['GG']+ pdf14['WS']+ pdf14['GEN']
pdf14

In [ ]:
pdf14 = pdf14.loc[:,['nintendo','playstation']]
pdf14

In [ ]:
import numpy as np

print(np.mean(pdf14['nintendo']))
print(np.mean(pdf14['playstation']))

In [ ]:
import scipy.stats

scipy.stats.ttest_ind(pdf14['nintendo'], pdf14['playstation'], equal_var=False)

# 출고량 많은 플랫폼, 게임 선정해서 출시연도랑 비교

In [ ]:
#출고량 많은 게임 top5 출시연도확인하기, 플랫폼 확인하기
pdf5 = pdf[['Name','Platform','Genre','Year','Total_Sales (unit:M)']]
pdf5.dropna(inplace=True)

In [ ]:
During = 2021 - pdf5['Year']
During

In [ ]:
pdf5['Sales_per_year'] = pdf5['Total_Sales (unit:M)']/ During
pdf5.sort_values('Sales_per_year', ascending=False).reset_index(drop=True)


In [ ]:
pdf5_P = pdf5.nlargest(10, 'Sales_per_year')
pdf5_P = pdf5_P.reset_index(drop=True)
pdf5_P

In [ ]:
pdf5_Pn = pdf5_P[['Name','Sales_per_year']]
pdf5_Pn

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
#plt.figure(figsize=(20,10))
plt.rcParams["figure.figsize"] = (10,6)
ax = sns.barplot(x = pdf5_P['Sales_per_year'], y = pdf5_P['Name'], data=pdf5_P, palette='coolwarm')

plt.xlabel('Total_Sales (unit:M)')#,fontsize = 20)
plt.ylabel("Games")#,fontsize = 20
#plt.xticks(fontsize = 20)
plt.title('< Top 10 > Sales per Year')#, fontsize=30)
#plt.xticks(rotation = 20) 각도 조절을 해도 이름이 다 들어오지 않아 x,y축 변경
plt.show();

# 출시년도부터 연평균 판매율 장르기준

In [ ]:
pdf5_Pg = pdf5[['Genre','Sales_per_year']]
pdf5_Pg

In [ ]:
pdf5_Pg = pdf5_Pg.groupby(by = ['Genre']).sum()
pdf5_Pg

In [ ]:
pdf5_Pg = pdf5_Pg.nlargest(10, 'Sales_per_year')
#
pdf5_Pg

In [ ]:
plt.rcParams["figure.figsize"] = (10,6)
ax = sns.barplot(x = pdf5_Pg['Sales_per_year'], y = pdf5_Pg.index, data=pdf5_Pg, palette='coolwarm')

plt.xlabel('Total_Sales (unit:M)')#,fontsize = 20)
plt.ylabel("Genre")#,fontsize = 20
#plt.xticks(fontsize = 20)
plt.title('<Top 10>Global Sales per year')#, fontsize=30)
#plt.xticks(rotation = 20) 각도 조절을 해도 이름이 다 들어오지 않아 x,y축 변경
plt.show();